# 準備

In [1]:
from dotenv import load_dotenv
load_dotenv() # .envファイルから環境変数を読み込み(OPENAI_API_KEY変数にAPI Keyが入っている)

True

# LLM Chain

In [2]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI() # これだけでモデルを初期化できるぽい

In [3]:
# これでモデルと会話することができる
print(llm.invoke("langsmithはどのようにテストを支援してくれますか？"))

content='langsmithはどのようにテストを支援してくれますか？'
content='Langsmithは、テストを支援するために以下のような機能を提供します。\n\n1. テストの作成: Langsmithは、テストケースを作成するための簡単なインターフェースを提供します。ユーザーは、入力値や期待される出力などの情報を指定し、テストケースを作成することができます。\n\n2. テスト実行: Langsmithは、作成したテストケースを実行するための機能を提供します。ユーザーは、テストの実行をトリガーし、結果を確認することができます。また、テストの実行結果のログや詳細なレポートも提供されます。\n\n3. テストの管理: Langsmithは、作成したテストケースや実行結果を管理するための機能を提供します。ユーザーは、テストの追加、編集、削除などを行うことができます。また、過去のテスト結果を参照することもできます。\n\n4. テストの自動化: Langsmithは、テストの自動化をサポートするための機能を提供します。ユーザーは、定期的なテストの実行や自動テストの設定を行うことができます。これにより、テストの効率化や品質向上を図ることができます。\n\n以上のような機能により、Langsmithはテストの作成、実行、管理、自動化などを支援します。'


In [4]:
# プロンプトテンプレートを使うこともできる
from langchain_core.prompts import ChatPromptTemplate

# この例だとシステムに「技術ドキュメントライター」というロールを与えている
prompt = ChatPromptTemplate.from_messages([
    ("system", "あなたは世界レベルの技術ドキュメントライターです。"),
    ("user", "{input}")
])

In [5]:
# 組み合わせてchainをつくる
chain = prompt | llm

In [6]:
# chainもinvokeで会話できる
chain.invoke({"input": "langsmithはどのようにテストを支援してくれますか？"})

content='あなたは世界レベルの技術ドキュメントライターです。'
content='langsmithはどのようにテストを支援してくれますか？'


AIMessage(content='langsmithは、テスト支援を以下のような方法で行います：\n\n1. テスト計画とテストケースの作成：langsmithは、テスト計画やテストケースの作成をサポートします。テスト計画やテストケースのテンプレートを提供し、テストの目的や範囲、実施手順、期待される結果などを明確に記述することができます。\n\n2. テスト実行のドキュメント化：テスト実行中に発生した問題やエラー、特に再現性の低いバグについては、langsmithが詳細なレポートを作成することができます。これにより、開発者が問題を修正するために必要な情報を提供し、効率的な問題解決を支援します。\n\n3. テスト結果のまとめと分析：テストの結果をまとめ、優先順位付けや傾向の分析を行うことができます。langsmithは、グラフやチャートを使用して結果を視覚化し、品質改善のための洞察を提供します。\n\n4. テストドキュメントの保守と更新：langsmithは、テストドキュメントの保守と更新をサポートします。テストケースやテスト計画の変更があった場合、langsmithは関連するドキュメントを自動的に更新し、一貫性を保つことができます。\n\n以上のような機能により、langsmithはテストプロセスを効率化し、品質向上に貢献します。')

In [7]:
# 出力をAIMessageからStringに変える
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [8]:
# chainに加える
chain = prompt | llm | output_parser

In [9]:
chain.invoke({"input": "langsmithはどのようにテストを支援してくれますか？"})

content='あなたは世界レベルの技術ドキュメントライターです。'
content='langsmithはどのようにテストを支援してくれますか？'


'Langsmithは、テストを支援するためにさまざまな方法を提供します。\n\n1. テスト計画の作成: Langsmithは、機能や要件に基づいてテスト計画を作成するのに役立ちます。テストの範囲、テストケース、テストデータ、テスト環境などを文書化することができます。\n\n2. テストスクリプトの作成: Langsmithは、テストスクリプトを作成するためのテンプレートやガイドラインを提供します。テストスクリプトは、テストの手順や期待される結果を明確に示すために使用されます。\n\n3. テスト結果の文書化: Langsmithは、テスト実行の結果を文書化するためのテンプレートやツールを提供します。テスト結果やエラーレポートを正確に記録し、問題を特定し解決するための情報を提供します。\n\n4. テストマニュアルの作成: Langsmithは、テストプロセスやテスト手法に関するマニュアルを作成するための支援を行います。テストの実施手順、テストツールの使用方法、テストデータの作成方法などを文書化することができます。\n\n5. テストレビューの支援: Langsmithは、テストドキュメントのレビューを支援します。他の関係者がテスト計画やテストスクリプトを確認し、フィードバックを提供できるようにします。\n\nLangsmithの自動化機能を活用することで、テストの自動化やテスト結果の自動生成も可能です。これにより、テストの効率性を向上させることができます。'

## Diving Deeper

さらに深く理解するためにたくさんのドキュメントが用意されている。今すぐにはとても読めないので、少しずつ読む。

# Retrieval Chain

In [10]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/overview") # URLを渡すとその内容を取ってきてくれる

docs = loader.load() # 指定したURLからドキュメントを取得する

In [11]:
from langchain_openai import OpenAIEmbeddings # ベクターストアに格納するためにドキュメントをベクトル化するためのモジュール

embeddings = OpenAIEmbeddings()

# これを使って、今からベクターストアにドキュメントを挿入する

In [12]:
# 簡単のため、シンプルなローカルベクターストアであるFAISSを使う
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter() # チャンクサイズの制限を下回るまで再帰的にテキストを分割するTextSplitter
documents = text_splitter.split_documents(docs) # TextSpritterに書けてテキストを分割する
vector = FAISS.from_documents(documents, embeddings) # 分割した文書をベクトル化してベクターストアに保存する

In [13]:
documents

[Document(page_content='LangSmith Overview and User Guide | 🦜️🛠️ LangSmith', metadata={'source': 'https://docs.smith.langchain.com/overview', 'title': 'LangSmith Overview and User Guide | 🦜️🛠️ LangSmith', 'description': 'Building reliable LLM applications can be challenging. LangChain simplifies the initial setup, but there is still work needed to bring the performance of prompts, chains and agents up the level where they are reliable enough to be used in production.', 'language': 'en'}),
 Document(page_content="Skip to main content🦜️🛠️ LangSmith DocsPython DocsJS/TS DocsSearchGo to AppLangSmithOverviewTracingTesting & EvaluationOrganizationsHubLangSmith CookbookOverviewOn this pageLangSmith Overview and User GuideBuilding reliable LLM applications can be challenging. LangChain simplifies the initial setup, but there is still work needed to bring the performance of prompts, chains and agents up the level where they are reliable enough to be used in production.Over the past two months, 

In [14]:
from langchain.chains.combine_documents import create_stuff_documents_chain 

# 与えたコンテキストに基づいて質問に回答させるためのプロンプト
prompt = ChatPromptTemplate.from_template("""与えられた文脈のみに基づいて質問に回答してください:
<context>
{context}
</context>

質問: {input}""")

# 質問と取得したドキュメントを受け取り、解答を生成するチェーンを作成する
# create_stuff_documents_chainはドキュメントのリストをコンテキストとしてプロンプトに渡す
document_chain = create_stuff_documents_chain(llm, prompt)

In [15]:
from langchain_core.documents import Document 

# ドキュメントを自分で渡すことで、チェーンを自分で実行することもできる
# document_chain.invoke({
#     "input": "how can langsmith help with testing?",
#     "context": [Document(page_content="langsmith can let you visualize test results")]
# })

In [16]:
# でも、retrieverを使うことで、質問に最も関連性の高いドキュメントを動的に選択してモデルに渡すことができる
from langchain.chains import create_retrieval_chain # retrieval chainを作るための関数

retriever = vector.as_retriever() # retrieverを作成
retrieval_chain = create_retrieval_chain(retriever, document_chain) # retrieverとドキュメントを受け取るchainからretrieval chainを作成

In [17]:
# これで、retrieval chainを呼び出すことができる
# response = retrieval_chain.invoke({"input": "LangSmithはどのようにテストを支援してくれますか?"})
# print(response["answer"])

# LangSmith offers several features that can help with testing:...

## Diving Deeper

これも少しずつ読む

# Conversation Retrieval Chain

ここまでやってきたのは1つの質問に答えるchain。でもこれだと一問一答で会話ができない。フォローアップクエスチョンにもこたえられるchainを作るにはどうすればいいだろうか。

In [18]:
# create_retrieval_chain()をまだ使うことができる。でも変更を加える必要がある
# 1. 直近の入力だけでなく、すべての会話の歴史を入力に加える
# 2. 最後のLLM chainも同様にすべての会話の歴史を受け取って出力を生成する

from langchain.chains import create_history_aware_retriever # 歴史を考慮したretriever
from langchain_core.prompts import MessagesPlaceholder # メッセージプレースホルダー (何なのかよくわからないぞ) 

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"), # メッセージプレースホルダの中にchat_historyを入れる
    ("user", "{input}"), # ユーザのインプット
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
    # 以上を踏まえて、会話に関係する情報を取得するために検索する検索クエリを生成してください
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt) # 歴史を踏まえた生成を行うchainを生成

In [19]:
from langchain_core.messages import HumanMessage, AIMessage # 人間のメッセージとAIのメッセージ

# 仮の歴史
chat_history = [HumanMessage(content="LangSmithは私のLLMアプリケーションのテストを支援してくれますか？"), AIMessage(content="もちろんです!")]

# chainに入力して続きを出力
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "どのように支援してくれるのですか"
})

# LLMはクエリを生成するため、LangSmithを使ったテストに関するドキュメントを返す(なんでクエリを生成したらドキュメントが返ってくるのかわかんない)

content='LangSmithは私のLLMアプリケーションのテストを支援してくれますか？'
content='もちろんです!'
content='どのように支援してくれるのですか'
content='Given the above conversation, generate a search query to look up in order to get information relevant to the conversation'


[Document(page_content='LangSmith Overview and User Guide | 🦜️🛠️ LangSmith', metadata={'source': 'https://docs.smith.langchain.com/overview', 'title': 'LangSmith Overview and User Guide | 🦜️🛠️ LangSmith', 'description': 'Building reliable LLM applications can be challenging. LangChain simplifies the initial setup, but there is still work needed to bring the performance of prompts, chains and agents up the level where they are reliable enough to be used in production.', 'language': 'en'}),
 Document(page_content="Skip to main content🦜️🛠️ LangSmith DocsPython DocsJS/TS DocsSearchGo to AppLangSmithOverviewTracingTesting & EvaluationOrganizationsHubLangSmith CookbookOverviewOn this pageLangSmith Overview and User GuideBuilding reliable LLM applications can be challenging. LangChain simplifies the initial setup, but there is still work needed to bring the performance of prompts, chains and agents up the level where they are reliable enough to be used in production.Over the past two months, 

In [20]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "以下のコンテキストを参照してユーザーの質問に答えてください:\n\n{context}"), # コンテキスト
    MessagesPlaceholder(variable_name="chat_history"), # 会話の歴史
    ("user", "{input}"), # ユーザーの入力
])
document_chain = create_stuff_documents_chain(llm, prompt) # 変更したプロンプトで再びchainを生成

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain) # 歴史も参照できるchainを生成

In [21]:
chat_history = [HumanMessage(content="LangSmithは私のLLMアプリケーションのテストを支援してくれますか?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "どのように支援してくれますか？"
})

content='LangSmithは私のLLMアプリケーションのテストを支援してくれますか?'
content='Yes!'
content='どのように支援してくれますか？'
content='Given the above conversation, generate a search query to look up in order to get information relevant to the conversation'
content='以下のコンテキストを参照してユーザーの質問に答えてください:\n\nSkip to main content🦜️🛠️ LangSmith DocsPython DocsJS/TS DocsSearchGo to AppLangSmithOverviewTracingTesting & EvaluationOrganizationsHubLangSmith CookbookOverviewOn this pageLangSmith Overview and User GuideBuilding reliable LLM applications can be challenging. LangChain simplifies the initial setup, but there is still work needed to bring the performance of prompts, chains and agents up the level where they are reliable enough to be used in production.Over the past two months, we at LangChain have been building and using LangSmith with the goal of bridging this gap. This is our tactical user guide to outline effective ways to use LangSmith and maximize its benefits.On by default\u200bAt LangChain, all of us have LangSmith’s t

{'chat_history': [HumanMessage(content='LangSmithは私のLLMアプリケーションのテストを支援してくれますか?'),
  AIMessage(content='Yes!')],
 'input': 'どのように支援してくれますか？',
 'context': [Document(page_content="Skip to main content🦜️🛠️ LangSmith DocsPython DocsJS/TS DocsSearchGo to AppLangSmithOverviewTracingTesting & EvaluationOrganizationsHubLangSmith CookbookOverviewOn this pageLangSmith Overview and User GuideBuilding reliable LLM applications can be challenging. LangChain simplifies the initial setup, but there is still work needed to bring the performance of prompts, chains and agents up the level where they are reliable enough to be used in production.Over the past two months, we at LangChain have been building and using LangSmith with the goal of bridging this gap. This is our tactical user guide to outline effective ways to use LangSmith and maximize its benefits.On by default\u200bAt LangChain, all of us have LangSmith’s tracing running in the background by default. On the Python side, this is achieved by set

# Agent

LLMがどのような手順を実行するのかを決定する

In [22]:
# まずはエージェントがどのツールにアクセスできるのかを定義する
from langchain.tools.retriever import create_retriever_tool # 今までのはcreate_retrieval_chainだった

retriever_tool = create_retriever_tool(
    retriever, # 会話の履歴を取ってくるもの
    "langsmith_search", # ツールの名前?
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!", # このツールの使い方をモデルに説明してる
    # ↑ LangSmithに関する情報を探せ。LangSmithに関する質問が来たら、このツールを使わなくてはなりませんよ！
)

In [23]:
# 検索ツールとしてTavilyを使う
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults() # エージェントが使う検索ツール

In [24]:
# エージェントが使うツールのリスト
tools = [retriever_tool, search]

In [25]:
from langchain_openai import ChatOpenAI # LLMモデルをロードするためのモジュール
from langchain import hub # プロンプトをpullするためのモジュール
from langchain.agents import create_openai_functions_agent # エージェントを作るためのモジュール
from langchain.agents import AgentExecutor # エージェントを作るためのモジュール

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent") # プロンプトをpullしてくる
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0) # モデルを指定してAPIをロード
agent = create_openai_functions_agent(llm, tools, prompt) # モデル、ツール、プロンプトをまとめて渡し、エージェントを初期化
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True) # エージェントを実行する準備を完了

In [26]:
# agent_executor.invoke({"input": "how can langsmith help with testing?"})

In [27]:
# agent_executor.invoke({"input": "what is the weather in SF?"})

In [28]:
from langserve import RemoteRunnable

remote_chain = RemoteRunnable("http://localhost:8000/agent/")

In [29]:
# 真嘉比さん: chat_historyはAI/HumanMessageのリストでもjsonでもどっちでもいい。Messageはjsonをラッピングしているに過ぎない
# このエラーは根が深く、ソースを見てもyamachanさんとペアプロしても解決できてないため、一旦飛ばす
chat_history = [{"content": "LangSmithはLLMアプリケーションのテストを支援してくれますか？", "type": "human"}, {"content": "もちろんです！", "type": "ai"}]
remote_chain.invoke({"input": "LangSmithって何ですか", "chat_history": chat_history})

result from serializer:{'input': 'LangSmithって何ですか', 'chat_history': [{'content': 'LangSmithはLLMアプリケーションのテストを支援してくれますか？', 'type': 'human'}, {'content': 'もちろんです！', 'type': 'ai'}]}


ConnectError: [WinError 10061] 対象のコンピューターによって拒否されたため、接続できませんでした。

In [ ]:
agent_executor.invoke({"input": "langsmithって何ですか？", "chat_history": chat_history})